In [3]:

from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
import h5py

def take_feature(Model_pre, image_size, lambda_f=None):
    width = image_size[0]
    height = image_size[1]
    inputs = Input((height, width, 3))
    if lambda_f:       #模型中有处理输入的情况
        inputs = Lambda(lambda_f)(inputs)
    
    gen = ImageDataGenerator()
    train_gen = gen.flow_from_directory("trainS", image_size, shuffle=False, 
                                              batch_size=16)
    test_gen = gen.flow_from_directory("testS", image_size, shuffle=False, 
                                             batch_size=16, class_mode=None)
    
    base = Model_pre(input_tensor=inputs, weights='imagenet', include_top=False)
    model = Model(base.input, GlobalAveragePooling2D()(base.output))
    
    train = model.predict_generator(train_gen)
    test = model.predict_generator(test_gen)
    with h5py.File("features_%s.h5"%Model_pre.__name__) as h:
        h.create_dataset("train", data=train)
        h.create_dataset("test", data=test)
        h.create_dataset("label", data=train_gen.classes)

take_feature(VGG19, (224, 224),vgg19.preprocess_input)
take_feature(InceptionResNetV2, (299, 299), inception_resnet_v2.preprocess_input)
take_feature(Xception, (299, 299), xception.preprocess_input)
'''
#def take_feature(Model_pre):
#    print("features_%s.h5"%Model_pre.__name__)
#take_feature(VGG19)
'''

Found 25000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.
Found 25000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.
Found 25000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.


'\n#def take_feature(Model_pre):\n#    print("features_%s.h5"%Model_pre.__name__)\n#take_feature(VGG19)\n'